#Trusted to refined: NBA

In [0]:
%run ./conexion_datalake

###Variables

In [0]:
tables = ["teams", "ranking", "players", "players_info", "games", "games_details", "players_salaries", "active_players"]

In [0]:
#Importe de funciones de sparksql
import pyspark.sql.functions as F

##Importe de tablas desde datalake y creacion de vistas temporales

In [0]:
for table in tables:
    exec(f'{table} = spark.read.parquet(f"abfss://{container}@{datalake}.dfs.core.windows.net/integrador/trusted_data/{table}.parquet")')
    exec(f'{table}.createOrReplaceTempView("{table}")')

###Join Players:

Tengo una tabla de jugadores activos, me quiero traer la informacion solo de esos jugadores, por lo tanto hago un INNER JOIN

In [0]:
 active_players_info = spark.sql("""
    SELECT *
    FROM active_players ap                            
    INNER JOIN players_info pi ON ap.id = pi.player_id
    """)

In [0]:
#borramos columnas repetidas
active_players_info = active_players_info.drop("full_name", "id")

In [0]:
#join players(tiene la info de los jugadores por temporada), lo uno con player_salaries(tiene la info del salario por temporada)

players_salaries = spark.sql("""
    SELECT p.player_name,
    p.player_id,
    p.team_id,
    p.season,
    ps.salary
    FROM players p
    LEFT JOIN players_salaries ps ON (p.player_name = ps.name AND p.season = ps.season)       
        """)

In [0]:
#hago join con active players, para quedarme solo con la info de los jugadores activos
players_salaries.createOrReplaceTempView("players_salaries")

In [0]:
players_salaries = spark.sql("""
    SELECT
    ps.player_id,
    ps.team_id,
    ps.season,
    ps.salary
    FROM players_salaries ps
    INNER JOIN active_players ap ON ps.player_id = ap.id       
        """)

##Games

In [0]:
games_game_details = spark.sql(
    """
    SELECT g.game_date_est, g.game_id, g.home_team_id, g.season, g.pts_home, g.fg_pct_home, g.ft_pct_home, g.fg3_pct_home, g.ast_home, g.reb_home, g.away_team_id, g.pts_away, g.fg_pct_away, g.ft_pct_away, g.fg3_pct_away, g.ast_away, g.reb_away, g.home_team_wins, g.winner_team_id, gd.team_id, gd.player_id, gd.fgm, gd.fga, gd.fg_pct, gd.fg3m, gd.fg3a, gd.fg3_pct, gd.ftm, gd.fta, gd.ft_pct, gd.oreb, gd.dreb, gd.reb, gd.ast, gd.stl, gd.blk, gd.to, gd.pf, gd.pts, gd.plus_minus, gd.min_played, gd.sec_played 
    FROM games g
    LEFT JOIN games_details gd ON g.game_id = gd.game_id
                  """
)

In [0]:
games_game_details.createOrReplaceTempView("games_game_details")

In [0]:
#ahora filtramos solo por los juegos en los cuales participan nuestros jugadores activos
games = spark.sql(""" 
    SELECT *
    FROM games_game_details ggd
    INNER JOIN active_players ap ON ggd.player_id = ap.id
    """)

In [0]:
#elimino columnas
games = games.drop("id", "is_active")


##Envio de Dataframes a DB

In [0]:
%run ./conexion_jdbc

In [0]:
teams.write.mode("overwrite")\
.format("jdbc")\
.option("driver", "com.microsoft.sqlserver.jdbc.SQLServerDriver")\
.option("url", f"jdbc:sqlserver://{jdbcHostname}:{jdbcPort};databaseName={jdbcDatabase};user={jdbcUsername};password={jdbcPassword}")\
.option("dbtable", "teams_NBA_ED")\
.save()

ranking.write.mode("overwrite")\
.format("jdbc")\
.option("driver", "com.microsoft.sqlserver.jdbc.SQLServerDriver")\
.option("url", f"jdbc:sqlserver://{jdbcHostname}:{jdbcPort};databaseName={jdbcDatabase};user={jdbcUsername};password={jdbcPassword}")\
.option("dbtable", "ranking_NBA_ED")\
.save()

active_players_info.write.mode("overwrite")\
.format("jdbc")\
.option("driver", "com.microsoft.sqlserver.jdbc.SQLServerDriver")\
.option("url", f"jdbc:sqlserver://{jdbcHostname}:{jdbcPort};databaseName={jdbcDatabase};user={jdbcUsername};password={jdbcPassword}")\
.option("dbtable", "players_NBA_ED")\
.save()

players_salaries.write.mode("overwrite")\
.format("jdbc")\
.option("driver", "com.microsoft.sqlserver.jdbc.SQLServerDriver")\
.option("url", f"jdbc:sqlserver://{jdbcHostname}:{jdbcPort};databaseName={jdbcDatabase};user={jdbcUsername};password={jdbcPassword}")\
.option("dbtable", "players_salaries_NBA_ED")\
.save()

games.write.mode("overwrite")\
.format("jdbc")\
.option("driver", "com.microsoft.sqlserver.jdbc.SQLServerDriver")\
.option("url", f"jdbc:sqlserver://{jdbcHostname}:{jdbcPort};databaseName={jdbcDatabase};user={jdbcUsername};password={jdbcPassword}")\
.option("dbtable", "games_NBA_ED")\
.save()